In [ ]:
# Crew Installation
%pip install crewai langchain
%pip install 'crewai[tools]'

In [ ]:
from langchain_core.language_models.llms import LLM
from langchain.schema import Generation, LLMResult
from langchain.callbacks.manager import CallbackManagerForLLMRun
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool

import os
import requests

LM_STUDIO_URL = "http://localhost:1234/v1/chat/completions"
# os.environ["SERPER_API_KEY"] = "Your Key" # serper.dev API key

class LMStudioLLM(LLM):
    @property
    def _llm_type(self) -> str:
        return "custom"

    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> LLMResult:
        generations = []
        for prompt in prompts:
            response = self.generate_content(prompt)
            generations.append([Generation(text=response)])
        return LLMResult(generations=generations)

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        return self.generate_content(prompt)

    def generate_content(self, prompt, max_tokens=100, temperature=0.7, model="lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf"):
        headers = {
            'Content-Type': 'application/json',
        }
        payload = {
            "model": model,
            "messages": [
                {"role": "user", "content": prompt}
            ],
            "max_tokens": max_tokens,
            "temperature": temperature,
        }
        
        try:
            response = requests.post(LM_STUDIO_URL, headers=headers, json=payload)
            response.raise_for_status()
            
            result = response.json()
            return result['choices'][0]['message']['content']
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")
            return ""

In [ ]:
# Define the model configuration
local_model = LMStudioLLM()
search_tool = SerperDevTool()

In [ ]:
# Define your agents with roles and goals
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
  verbose=True,
  allow_delegation=False,
  llm=local_model,
  tools=[search_tool],
  max_iterations=15,
  max_execution_time=600
)

writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=True,
  llm=local_model,
  max_iterations=15,
  max_execution_time=600
)

In [ ]:
# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
  expected_output="Full blog post of at least 4 paragraphs",
  agent=writer
)

In [ ]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=2,
)

In [ ]:
# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)